In [32]:
def recommend_airbnbs(filepath="../ML-exam/data/clustered_airbnb.csv"):
    import pandas as pd

    # Load data
    df = pd.read_csv(filepath)

    # --- Available cities ---
    cities = sorted(df['City'].unique())
    print("Available cities:")
    for c in cities:
        print("-", c)

    # --- User Inputs ---
    print("\n--- Enter your preferences ---")
    try:
        user_budget = float(input("Total budget (€): "))
        duration_input = input("Number of nights (leave blank to maximize duration): ").strip()
        min_bedrooms = int(input("Minimum number of bedrooms: "))
        max_city_dist = float(input("Max distance to city center (e.g. 3.0): "))
        max_metro_dist = float(input("Max distance to metro (e.g. 1.0): "))
        weekend = input("Is your stay during a weekend? (yes/no): ").lower() == "yes"
        city_input = input("Pick a city (leave blank to search all): ").strip()
    except ValueError:
        print("Invalid input. Please try again.")
        return

    # Determine mode
    if duration_input == "":
        mode = "duration"
        user_duration = None
    else:
        mode = "value"
        try:
            user_duration = int(duration_input)
        except ValueError:
            print("Invalid number of nights. Please enter an integer.")
            return

    # --- Filter listings ---
    filtered_df = df.copy()
    if weekend:
        filtered_df = filtered_df[filtered_df['Is_weekend_bool'] == 1]
    if city_input:
        filtered_df = filtered_df[filtered_df['City'].str.lower() == city_input.lower()]
    filtered_df = filtered_df[
        (filtered_df['bedrooms'] >= min_bedrooms) &
        (filtered_df['dist'] <= max_city_dist) &
        (filtered_df['metro_dist'] <= max_metro_dist)
    ].copy()

    if filtered_df.empty:
        print("\nNo listings match your criteria.")
        return

    # --- Compute recommendations ---
    if mode == "duration":
        filtered_df['max_nights'] = (user_budget / filtered_df['realSum']).apply(int)
        recommended = filtered_df.sort_values(by='max_nights', ascending=False)
        display_cols = ['City', 'realSum', 'bedrooms', 'dist', 'metro_dist',
                        'guest_satisfaction_overall', 'max_nights']
    else:
        filtered_df['price_per_day'] = filtered_df['realSum'] / user_duration
        filtered_df['value_score'] = filtered_df['guest_satisfaction_overall'] / filtered_df['price_per_day']
        recommended = filtered_df.sort_values(by='value_score', ascending=False)
        display_cols = ['City', 'realSum', 'bedrooms', 'dist', 'metro_dist',
                        'guest_satisfaction_overall', 'value_score']

    # --- Output Results ---
    print(f"\nTop 10 Recommended Listings (Mode: {mode}):\n")
    print(recommended[display_cols].head(10).to_string(index=False))


In [ ]:
recommend_airbnbs()


Available cities:
- Amsterdam
- Athens
- Barcelona
- Berlin
- Budapest
- Lisbon
- London
- Paris
- Rome
- Vienna

--- Enter your preferences ---
